# Fitting the Roitman Shadlen (2002) data using PyDDM
In this example, we load data from the open dataset by Roitman and Shadlen (2002). This dataset can be [downloaded here](https://shadlenlab.columbia.edu/resources/RoitmanDataCode.html). We use a preprocessed version below.

In [ ]:
#@title Prepare the environment
!pip -q install git+https://github.com/mwshinn/PyDDM


In [ ]:
from ddm import Sample
import pandas
df_rt = pandas.read_csv("https://raw.githubusercontent.com/mwshinn/PyDDM/master/doc/downloads/roitman_rts.csv")
df_rt

Let's fit only monkey 1 for right now.  We will filter out the shortest and longest RTs as they did in Roitman and Shadlen (2002), but this is not necessary.  Finally, we use these data to create a "sample" object for PyDDM.  For each trial, it will contain the RT, whether the response was correct, and three conditions: the coherence ("coh"), the monkey ("monkey", here always 1), and whether they chose the left or right target ("trgchoice").

In [ ]:

df_rt = df_rt[df_rt["monkey"] == 1] # Only monkey 1
  
# Remove short and long RTs, as in 10.1523/JNEUROSCI.4684-04.2005.
# This is not strictly necessary, but is performed here for
# compatibility with this study.
df_rt = df_rt[df_rt["rt"] > .1] # Remove trials less than 100ms
df_rt = df_rt[df_rt["rt"] < 1.65] # Remove trials greater than 1650ms
  
# Create a sample object from our data.  This is the standard input
# format for fitting procedures.  Since RT and correct/error are
# both mandatory columns, their names are specified by command line
# arguments.
roitman_sample = Sample.from_pandas_dataframe(df_rt, rt_column_name="rt", correct_column_name="correct")

# Fitting a DDM using PyDDM
First, we want to let the drift rate vary with the coherence. To do so, we must subclass Drift. Each subclass must contain a name (a short description of how drift varies), required parameters (a list of the parameters that must be passed when we initialize our subclass, i.e. parameters which are passed to the constructor), and required conditions (a list of conditions that must be present in any data when we fit data to the model). We can easily define a model that fits our needs:

In [ ]:
import ddm.models
class DriftCoherence(ddm.models.Drift):
    name = "Drift depends linearly on coherence"
    required_parameters = ["driftcoh"] # <-- Parameters we want to include in the model
    required_conditions = ["coh"] # <-- Task parameters ("conditions"). Should be the same name as in the sample.
    
    # We must always define the get_drift function, which is used to compute the instantaneous value of drift.
    def get_drift(self, conditions, **kwargs):
        return self.driftcoh * conditions['coh']

Because we are fitting with likelihood, we must include a baseline lapse rate to avoid taking the log of 0. Traditionally this is implemented with a uniform distribution, but PyDDM can also use an exponential distribution using OverlayPoissonMixture (representing a Poisson process lapse rate), as we use here. However, since we also want a non-decision time, we need to use two Overlay objects. To accomplish this, we can use an OverlayChain object. Then, we can construct a model which uses this and fit the data to the model:

In [ ]:
from ddm import Model, Fittable
from ddm.functions import fit_adjust_model, display_model
from ddm.models import NoiseConstant, BoundConstant, OverlayChain, OverlayNonDecision, OverlayPoissonMixture
model_rs = Model(name='Roitman data, drift varies with coherence',
                 drift=DriftCoherence(driftcoh=Fittable(minval=0, maxval=20)),
                 noise=NoiseConstant(noise=1),
                 bound=BoundConstant(B=Fittable(minval=.1, maxval=1.5)),
                 # Since we can only have one overlay, we use
                 # OverlayChain to string together multiple overlays.
                 # They are applied sequentially in order.  OverlayNonDecision
                 # implements a non-decision time by shifting the
                 # resulting distribution of response times by
                 # `nondectime` seconds.
                 overlay=OverlayChain(overlays=[OverlayNonDecision(nondectime=Fittable(minval=0, maxval=.4)),
                                                OverlayPoissonMixture(pmixturecoef=.02,
                                                                      rate=1)]),
                 dx=.001, dt=.01, T_dur=2)

# Fitting this will also be fast because PyDDM can automatically
# determine that DriftCoherence will allow an analytical solution.
fit_model_rs = fit_adjust_model(sample=roitman_sample, model=model_rs, verbose=False)

Now let's view the fitted model

In [ ]:
import ddm.plot
ddm.plot.model_gui_jupyter(model=fit_model_rs, sample=roitman_sample)

In [ ]:
display_model(fit_model_rs)

# Fitting a GDDM using PyDDM
Let’s see if we can improve the fit by including additional model components. We will include exponentially collapsing bounds and use a leaky or unstable integrator instead of a perfect integrator.

To use a coherence-dependent leaky or unstable integrator, we can build a drift model which incorporates the position of the decision variable to either increase or decrease drift rate. This can be accomplished by making get_drift depend on the argument x.  We don't need to define collapsing bounds, because this is included by default in PyDDM.

In [ ]:
class DriftCoherenceLeak(ddm.models.Drift):
    name = "Leaky drift depends linearly on coherence"
    required_parameters = ["driftcoh", "leak"] # <-- Parameters we want to include in the model
    required_conditions = ["coh"] # <-- Task parameters ("conditions"). Should be the same name as in the sample.
    
    # We must always define the get_drift function, which is used to compute the instantaneous value of drift.
    def get_drift(self, x, conditions, **kwargs):
        return self.driftcoh * conditions['coh'] + self.leak * x

Thus, the full model definition is

In [ ]:
from ddm.models import BoundCollapsingExponential
model_leak = Model(name='Roitman data, leaky drift varies with coherence',
                   drift=DriftCoherenceLeak(driftcoh=Fittable(minval=0, maxval=20),
                                            leak=Fittable(minval=-10, maxval=10)),
                   noise=NoiseConstant(noise=1),
                   bound=BoundCollapsingExponential(B=Fittable(minval=0.5, maxval=3),
                                                    tau=Fittable(minval=.0001, maxval=5)),
                   # Since we can only have one overlay, we use
                   # OverlayChain to string together multiple overlays.
                   # They are applied sequentially in order.  OverlayDelay
                   # implements a non-decision time by shifting the
                   # resulting distribution of response times by
                   # `delaytime` seconds.
                   overlay=OverlayChain(overlays=[OverlayNonDecision(nondectime=Fittable(minval=0, maxval=.4)),
                                                  OverlayPoissonMixture(pmixturecoef=.02,
                                                                        rate=1)]),
                   dx=.01, dt=.01, T_dur=2)

Before we fit the model, let's take a look at what these parameters do.

In [ ]:
ddm.plot.model_gui_jupyter(model=model_leak, sample=roitman_sample)

The function for fitting the model is shown below.  However, because this can take a few minutes (and is especially slow when running on Google Colab), we instead show a version which we already fit to data.

In [ ]:
# fit_model_lk = fit_adjust_model(sample=roitman_sample, model=model_leak)
fit_model_lk = Model(drift=DriftCoherenceLeak(driftcoh=10.718, leak=-.94),
                     noise=NoiseConstant(noise=1),
                     bound=BoundCollapsingExponential(B=1.557, tau=1.906),
                     overlay=OverlayChain(overlays=[OverlayNonDecision(nondectime=.2388),
                                                    OverlayPoissonMixture(pmixturecoef=.02, rate=1)]),
                     dx=.01, dt=.01, T_dur=2)

As before, we can visualize this model.  Note how this GDDM is a much better fit than the DDM we fit previously.

In [ ]:
ddm.plot.model_gui_jupyter(model=fit_model_lk, sample=roitman_sample)

In [ ]:
display_model(fit_model_rs)

# Conclusion
Hopefully this tutorial helped you see how you could use PyDDM and the GDDM in your own work.  See the documentation for more information, including the [Cookbook](https://pyddm.readthedocs.io/en/latest/cookbook/index.html), which contains a collection of plug-and-play model components you can use to build your own GDDM.